# Soheil HajianManesh

## Artificial Intelligence - CA1 - Genetic Algorithm

### In this computer assignment we must solve curve fitting problem using genetic algorithm.

At the end we must find best possible coefficients as an answer.
I consider each coefficient as a gene.
Each chromosome has n gene that n is equal to the number of coefficients (degree+1).
In the chrosomes coefficients are arranged according to their position in the equation(gene[0] coefficient of the largest term).

In [1]:
import random
import copy
from dataclasses import dataclass

`Consts` class consist of const parameter in my algorithm :

- `maxGenerationNum`:Maximum number of generation that we can generate in our algorithm to reach best solution.
- `PopulationNumber`:The total number of all chromosome (population size).
- `crossOverP`:The probability of crossover performed on two chromosome or they directly go to next generation.
- `mutationP`:The probability of mutation performed on a chromosome or chromosome go to next generation without changing.
- `directTransferPerc`:How many percentages of best chromosomes should be transferred directly to the next generation.
- `ChangePilotPerc`:The probability that specifies a gene must be change or no (in second cross over algorithm)

In [98]:
@dataclass
class Consts:
    maxGenerationNum: int
    PopulationNumber: int
    crossOverP: float
    mutationP: float
    directTransferPerc: float
    ChangePilotPerc: float
Gene= int
Chromosome=list[Gene]
Population=list[Chromosome]

In [3]:
class Point:
    x: int
    y: int
    def __init__(self,x: int,y: int):
        self.x=x
        self.y=y

`Input` class consist input fields such as:
- `degree`: The degree of the equation .
- `minCoef`: The minimum coefficient that the equation can have.
- `maxCoef`: The maximum coefficient that the equation can have
- `points` : A number of points are given as input.(Their number is arbitrary)

In [4]:
@dataclass 
class Input:
    degree: int
    minCoef: int
    maxCoef: int 
    points: list[Point]

## CurveFitter Class:
### - `createFirstPopulation`:
In this method create first population with size=`popSize` and each gene get random   number between `minCoef` and `maxCoef`


### - `calcFitness`:
I defined fitness function as fallows:
$$ \frac{1}{1 + \sum_{i=1}^{len(points)}|y^{'}(x_{i}) - y(x_{i})|}$$

for every chromosome loop over all points and define yprim - y and then calculate fitness
- Fitness is between 0 and 1.
- The closer the chromosome to right answer , the higher the fitness and it is closer to 1. 
- if one chromosome is contain right coefficients then $${\sum_{i=1}^{len(points)}|y^{'}(x_{i}) - y(x_{i})|}=0$$ and fitness become 1.


### - `createCarryPop`:
This method return `directTransferPerc*PopulationNumber` of first best chromosome named carrry population to directly transfer to the next generation.


### - `createMatingPool`:
In this method I create mating pool using `Rank-Based Selection`, Assign a rank to each chromosome in descending order;I mean 
for example first cromosome that is also the best chromosome give the biggest rang and the last chromosome gets 1 as rank.
then make new poplation that every chromosome have n member in new population `(n=rank)`,At the end shuffle new population and return first `PopulationNumber` of it as mating pool.

### - `createCrossOverPool`:
To create modified population if random number is under crossOverP we cross over two chromosome else we transfer two chromosome directly to modified population.for combine two parent chromosome and make two new child I use two algorithm that can use any of them you want.
   #### - `crossOverAlgorithm1`:
using `1-point crossover` algorithm .
#### - `crossOverAlgorithm2`: 
using `n-random-point crossover` algorithm .In this algorithm I loop over one parent and for each gene copy parent1 in child1 and parent2 in child2 until random number become less than `ChangePilotPerc`,then change the order and copy until random number become less than crossOverP again and change the order again.Do this until reach the end of the Chromosome.

### - `createMutationPool`:
In this method my algorithm for mutation is to for each chromosome if random number is less than mutationP then randomly choose one gene in chromosome and change its value to a random integar between minCoef and maxCoef.

### - `findCurve`:
At last this method does the process of finding `curve`:

- After initial first population every time in the cycle do these steps:
    - Shuffle population
    - Calculate fitness for each chromosome
    - If a fitness equal to 1 break the cycle and return answer
    - Sort population based on fitnesses
    - Specify carry population that directly transfer to next generation
    - Create mating pool using sorted population
    - Cross over mating pool
    - Mutation on crossOverPool
    - Combine mutationPool & carry population
    - Try this cycle `maxGenerationNum` time.

In [117]:
class CurveFitter:
    
    inputs:Input
        
        
    def __init__(self, inputs:Input):
        inputs=inputs

        
    def createFirstPopulation(self,popSize)->Population:
        population: Population=[None]*popSize
        for i in range(popSize):
            chromosome: Chromosome=[None]*(inputs.degree+1)
            chromosome[0::1]=[random.randint(inputs.minCoef,inputs.maxCoef) for _ in range(inputs.degree+1)]
            population[i]=chromosome
        return population
    
    
    def calcFitness(self,chromosome: Chromosome)->float:
        diffSummation=0
        for point in inputs.points:
            power=inputs.degree
            yprim=0
            for i in range(len(chromosome)):
                yprim+=chromosome[i]*(pow(point.x,power))
                power-=1
            diffSummation+=abs(yprim-point.y)
        fitness=1/(1+diffSummation)
        return fitness
    
    
    def createCarryPop(self,sortedPop: Population,size: int)->Population:
        carryChromosomes: Population = [None]*size
        for i in range(size):
            carryChromosomes[i]=sortedPop[i]
        return carryChromosomes
    
    
    def createMatingPool(self,sortedPop: Population,popSize: int)-> Population:
        ranks = list(reversed(range(1,popSize+1)))
        matingPool: Population = []
        for i in range(popSize):
            for _ in range(ranks[i]):
                matingPool.append(sortedPop[i])
        random.shuffle(matingPool)
        return matingPool[0:popSize]
    
    ###### Start Cross Over ######

    def createCrossOverPool(self,crossOverP: float,matingPool: Population,fitnesses: list[float],ChangePilotPerc: float)->Population:
        crossPool:Population=[]
        for i in range(0,len(matingPool),2):
            rand=random.random()
            if rand<crossOverP:
#                 children=self.crossOverAlgorithm1(matingPool[i],matingPool[i+1])
                children=self.crossOverAlgorithm2(matingPool[i],matingPool[i+1],ChangePilotPerc)
                crossPool.extend(children)
            else:
                crossPool.append(matingPool[i])
                crossPool.append(matingPool[i+1])
        return crossPool
    def crossOverAlgorithm1(self,parent1: Chromosome,parent2: Chromosome)->Population:
        children : Population = []
        crossIndex=random.randint(0,inputs.degree)
        children.append(parent1[0:crossIndex]+parent2[crossIndex:])
        children.append(parent2[0:crossIndex]+parent1[crossIndex:])
        return children

    def crossOverAlgorithm2(self,parent1: Chromosome,parent2: Chromosome,ChangePilotPerc: float)->Population:
        children : Population = []
        child1: Chromosome = [None] * len(parent1)
        child2: Chromosome = [None] * len(parent2)
        flag=1
        for i in range(len(parent1)):
            if random.random()<ChangePilotPerc:
                flag=flag*(-1)
            if flag==1:
                child1[i]=parent1[i]
                child2[i]=parent2[i]
            else:
                child1[i]=parent2[i]
                child2[i]=parent1[i]
        children.append(child1)
        children.append(child2)
        return children
    
    ###### END Cross Over ######
    ###### START Mutation ######
    
    def createMutationPool(self,mutationP: float,crossOverPool: Population)->Population:
        mutationPool:Population = copy.deepcopy(crossOverPool)
        for chromosome in mutationPool:
            if random.random()<mutationP:
                idx=random.randint(0,len(chromosome)-1)
                chromosome[idx]=random.randint(inputs.minCoef,inputs.maxCoef)
        return mutationPool
    
    ###### END Mutation ######
    
    def findCurve(self,consts:Consts)->Chromosome:
        population=self.createFirstPopulation(consts.PopulationNumber)
        generation=0
        for i in range(consts.maxGenerationNum):
            generation+=1
            random.shuffle(population)
            fitnesses=[self.calcFitness(chromo) for chromo in population]
            if(1 in fitnesses):
                return population[fitnesses.index(1)]
            sortedZip=sorted(zip(fitnesses,population),reverse=True,key=lambda pair: pair[0])
            sortedPopulation=[item[1] for item in sortedZip]
            directCarryCount=int(consts.PopulationNumber*consts.directTransferPerc)
            carryPopulation=self.createCarryPop(sortedPopulation,directCarryCount)
            matingPool=self.createMatingPool(sortedPopulation,consts.PopulationNumber)
            crossOverPool=self.createCrossOverPool(consts.crossOverP,matingPool,fitnesses,consts.ChangePilotPerc)
            mutationPool=self.createMutationPool(consts.mutationP,crossOverPool)
            population=mutationPool[0:(consts.PopulationNumber-directCarryCount)]
            population.extend(carryPopulation)
        return population[0]        
    

# TestCase1 example 

In [118]:
consts = Consts(
    maxGenerationNum = 2000,
    PopulationNumber = 400,
    crossOverP = 0.75,
    mutationP = 0.4,
    directTransferPerc = 0.1,
    ChangePilotPerc=0.2
)
inputs= Input(
    degree = 3,
    minCoef = -9,
    maxCoef = 9,
    points = [Point(0,1),Point(1,0),Point(2,-5),Point(-1,-8)]
)
curveFitter=CurveFitter(inputs)
resultCurve=curveFitter.findCurve(consts)
resultCurve

[1, -5, 3, 1]

# TestCase2 example 

In [119]:
consts = Consts(
    maxGenerationNum = 2000,
    PopulationNumber = 400,
    crossOverP = 0.75,
    mutationP = 0.4,
    directTransferPerc = 0.1,
    ChangePilotPerc=0.2
)
inputs= Input(
    degree = 6,
    minCoef = -9,
    maxCoef = 9,
    points = [Point(-2,1),Point(-1,-18),Point(0,-1),Point(1,10),Point(2,237),Point(3,2486),Point(-3,1202)]
)
curveFitter=CurveFitter(inputs)
resultCurve=curveFitter.findCurve(consts)
resultCurve

[3, 2, -4, 5, -2, 7, -1]

# Questions

1. *The problems caused by an extremely large or small population size:*  
    - if we have extremely large population it takes too much time and memory although we have more diversity and cause to find optimal solution in fewer generation.
    - if have have extremely small population obviously we have less diversity and maybe never reach to optimal solution.
 

2. *The effects of a growing population on the algorithm:*  
    - as mention in question 1 increase population size in each generation helps us because we have more diversity but it also consume more time and memory in each generation that is not good.
    - another bad point is we try to remove bad chromosome over generation and trying to converge our population to optimal answer but growing population in each generation harm the convergence.
3. *Compare effects of crossover and mutation.Can use only one of them?why?*  
- CrossOver :
    - Crossover is the process of combining genetic information from two parent chromosome to make better chromosomes
    - Crossover helps maintain diversity and it allows the algorithm to converge faster toward promising solutions by exploiting the information shared among the population.
- Mutation :
    - Mutation is the process of changing gene(s) of one chromosome to make a better one.
    - Mutation prevents premature convergence by ensuring that the search does not get stuck in local optima.
- While it is theoretically possible to use only one of these operators, doing so may have significant drawbacks:
   - 1.Using Only Crossover:  
   The GA may suffer from a lack of genetic diversity, leading to premature convergence to suboptimal solutions and get stuck in local optima
    - 2.Using Only Mutation:
     It may take a longer time to reach good solutions and maybe cause algorithm has less diversity.
4. *Approaches to solving the curve fitting problem faster:* 
    The parameters in problem are so effetive in solving speed.we can easily play with the and reach best values to solving problem faster.(change parameters in Const class) 
  
5. *Solutions to stagnated chromosomes and getting stuck:*  
    mutation is the solution for this problem as I mentioned in question 3 mutation helps us to escape local optima but if despite mutation still getting stuck , we can use `Multi-starting` means re-running algorithm with different initial population

6. *Stopping the algorithm if a solution does not exist:*  
      We can limit the number of generation and stop algorithm if reach that number and return chromosome with best fitness as
      result.(as I did in my code)
7. *As the polynomial degree increases, how does the time to find the coefficients change?* 
   When degree increases,out state space for chromosomes increase increases exponentially.Suppose that we don't have limit for number of generation,in worst case we have to traverse the entire state space to reach the answer so time to find the coefficients increases exponentially.


8. *effects of increase or decrease the number of points on algorithm:*  
    it depends on degree of equation :
    - If number of points is less than the limit to represent a single polynomial, then the smaller the number of points, the sooner the algorithm will reach the solution.because more polynomials include those points.
    - otherwise if number of points is reach the limit to represent a single polynomial, then the bigger the number of points, the sooner the algorithm will reach the solution.because we can calculate fitness more accurate for chromosmes.
